<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#DSPT6---Unit-3-Module-2---Consuming-Data-from-an-API" data-toc-modified-id="DSPT6---Unit-3-Module-2---Consuming-Data-from-an-API-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>DSPT6 - Unit 3 Module 2 - Consuming Data from an API</a></span></li><li><span><a href="#SpaCy-Embeddings" data-toc-modified-id="SpaCy-Embeddings-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>SpaCy Embeddings</a></span></li><li><span><a href="#Bringing-it-all-together" data-toc-modified-id="Bringing-it-all-together-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Bringing it all together</a></span></li></ul></div>

In [ ]:
### DSPT6 - Unit 3 Module 2 - Consuming Data from an API

'''The purpose of this notebook is to demonstrate:
- Connect to the Twitter API (and twitter_scraper) to query for tweets and user information by various parameters
- Convert tweet text using SpaCy into numerical embeddings that can be use in a predictive model'''

In [ ]:
import tweepy

In [ ]:
TWITTER_CONSUMER_API_KEY = 'pcoysZJR6Jr861alyjrVsllGZ'
TWITTER_CONSUMER_API_SECRET = 'Mv1ugepKLjPLBcTFMYuLkGfbDQcqWq6h05JavyMHka0SpKjngB'
TWITTER_ACCESS_TOKEN = '1306740878229331968-QtWPMuBomcuQfDixPTYmASWmdroAmb'
TWITTER_ACCESS_TOKEN_SECRET = 'XqtFK8hcfE55VimX9pQ94OyjXciDofd7GxOBVjXb0646N'

In [ ]:
TWITTER_AUTH = tweepy.OAuthHandler(TWITTER_CONSUMER_API_KEY, TWITTER_CONSUMER_API_SECRET)
TWITTER_AUTH.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)
TWITTER = tweepy.API(TWITTER_AUTH)

In [ ]:
dir(TWITTER)

In [ ]:
twitter_user = TWITTER.get_user('elonmusk')
twitter_user

In [ ]:
twitter_user.id

In [ ]:
elon_tweets = twitter_user.timeline()
elon_tweets

In [ ]:
elon_tweets[0].text

In [ ]:
elon_tweets = twitter_user.timeline(count=200,
                                    exclude_replies=True,
                                    include_rts=False,
                                    max_id='1303802636827799555',
                                    tweet_mode='extended')
len(elon_tweets)

In [ ]:
len(elon_tweets)

In [ ]:
elon_tweets[11].full_text

In [ ]:
elon_tweets[19].id

### SpaCy Embeddings

In [ ]:
import spacy

In [ ]:
nlp = spacy.load('en_core_web_md', disable=['tagger', 'parser'])

In [ ]:
tokens = nlp("dog cat banana daslkfjsalkjf")

for token in tokens:
    print(token.text, token.has_vector, token.is_oov)

In [ ]:
dog = nlp('dog')
dog.vector

In [ ]:
tweet_embedding = nlp(elon_tweets[0].full_text)
tweet_embedding.vector

### Bringing it all together

In [ ]:
def vectorize_tweet(nlp, tweet_text):
    return nlp(tweet_text).vector

In [ ]:
def add_or_update_user(username, nlp):
    try:
        twitter_user = TWITTER.get_user(username)

        tweets = twitter_user.timeline(count=200,
                                    exclude_replies=True,
                                    include_rts=False,
                                    tweet_mode='extended')

        embeddings = vectorize_tweet(nlp, tweets[0].full_text)
    
    except Exception as e:
        print('Error processing {}: {}'.format(username, e))

    return tweets, embeddings